Package Installation

In [ ]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib
!pip install mpl_finance
from google.colab import drive
drive.mount('/content/drive')

Selecting previously unselected package libta-lib0.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 5.9MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437791 sha256=19223b02b747915cb680cfde95a3e59e161d859381308bf0acf75528261b2848
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib
Mounted at /

Stock Number

In [ ]:
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002",
            "2105", "2207", 
            # "2227", 
            "2301", "2303", "2308", "2317", "2327",
            "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610",
            "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
            "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711",
            "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

GPU Environment Setting

In [ ]:
import tensorflow as tf
from tensorflow.keras import *
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    # tf.config.experimental.set_virtual_device_configuration(gpu0,
    # [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 


Import Package

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import talib
import pandas_datareader as pdr
import mpl_finance as mpf
import seaborn as sns
import datetime as datetime
from sklearn import preprocessing
import requests
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import csv
import statistics
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D,MaxPooling1D
from tensorflow.keras.layers import Activation,Dense ,Dropout ,GRU, ConvLSTM2D ,LSTM ,Bidirectional,TimeDistributed,Flatten, Conv1D, MaxPooling1D,BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers 
from tensorflow.keras.layers import LayerNormalization
from tensorflow.python.keras.layers import Input, Embedding, Dot, Reshape, Dense
from tensorflow.python.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adamax, Adam
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

Exponential Smoothing + One-Hot Encoding

In [ ]:
def exponential_smoothing(days, alpha, numdata):
  # assign weights
  weights = []
  i = days 
  while i>0:
    weights.append( math.exp(-alpha*i) )
    i-=1
  weights = np.array(weights) / sum(weights)
  # return data after smoothing
  results = []
  results += numdata[:days]
  for i in range(days, len(numdata)):
    results.append((np.dot( np.array(weights), np.array(numdata[i-days:i]))))
  return results

classes =['0', '1']
def to_onelist(text):
    onehot = [0] * 2
    onehot[ int(classes.index(str(text)) ) ] = 1
    label_list = np.array(onehot)
    return label_list

Preparation for data collection

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
start = '2005/01/01'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [ ]:
# for stockno in stockNum:
#     # load 進 五大價格 一個量
#     df = pdr.DataReader(str(stockno)+'.TW', 'yahoo', start=start, end=end)
#     df.index = df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
#     # print(df.columns)
#     stock_time = str(df.index[0]).split(" ")[0]
#     #load in 融資融券
#     url = "https://api.finmindtrade.com/api/v3/data"
#     parameter = {
#         "dataset": "TaiwanStockMarginPurchaseShortSale",
#         "stock_id": str(stockno),
#         "date": stock_time,
#     }
#     data = requests.get(url, params=parameter)
#     data = data.json()
#     data = pd.DataFrame(data['data'])
#     #if 沒資料 則跳過這張股票
#     if data.empty == True:
#         print(str(stockno)+" can't find data !")
#         continue
#     ratio = []
#     b_total = []
#     s_total = []
#     for i in range(len(data)):
#         b_total.append(data['MarginPurchaseLimit'][i] - data['MarginPurchaseTodayBalance'][i])
#         s_total.append(data['ShortSaleLimit'][i] - data['ShortSaleTodayBalance'][i])
#         ratio.append(float(data['ShortSaleLimit'][i] - data['ShortSaleTodayBalance'][i])/(data['MarginPurchaseLimit'][i] - data['MarginPurchaseTodayBalance'][i]+1))
#     data['b_total'] = b_total
#     data['s_total'] = s_total
#     data['b_s_ratio'] = ratio
#     data = data.set_index('date')
#     #合併這兩個表格
#     new_df = df.join(data)
#     new_df = new_df.drop(['stock_id'], axis=1)
#     new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
#     #處理三大法人投資情況 視情況加入
#     url = "https://api.finmindtrade.com/api/v3/data"
#     parameter = {
#         "dataset": "InstitutionalInvestorsBuySell",
#         "stock_id": str(stockno),
#         "date": stock_time,
#     }
#     data = requests.get(url, params=parameter)
#     data = data.json()
#     data = pd.DataFrame(data['data'])
#     if data.empty == True:
#         print(str(stockno)+" can't find data !")
#         continue
#     df = pd.DataFrame()
#     z = 0 
#     i = 0
#     while i < len(data)  :
#         a = data['date'][i]
#         count = 0
#         sell  = 0
#         buy = 0 
#         if i+6 < len(data)-1:
#             for j  in range(i, i+6):
#                 if data['date'][j]== a :
#                     count += 1
#                     sell += data['sell'][j]
#                     buy += data['buy'][j]
#         else:
#             for j in range(i, len(data)):
#                 if data['date'][j] == a :
#                     count += 1
#                     sell += data['sell'][j]
#                     buy += data['buy'][j]
#         df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
#         i+=count
#     # combine
#     new_df = new_df.join(df.set_index('date'))
#     new_df = new_df.dropna(axis=0,how='any')
#     new_df.reset_index(drop=True, inplace=True)
#     new_df = new_df.drop(['Note'], axis=1)
#     for columnName in ['Close','High', 'Low','Open']:
#         if str(columnName) == 'y_label' : continue
#         new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
#     pred_days = 5
#     ans = []
#     for i in range(len(new_df)-pred_days):
#         Pi = new_df['Close'][i+pred_days]
#         Pj = new_df['Close'][i]
#         if Pi > Pj : y = 1
#         else: y = 0
#         ans.append(y)
#     ans += [0]*pred_days
#     new_df['y_label']= ans
    
#     #store file
#     new_df = new_df[5:]
#     from google.colab import files
#     new_df.to_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5(for_price)/'+str(stockno)+'.csv')
#     # from google.colab import files
#     # files.download('/content/drive/My_Drive/Program_Trading_Code/0050_stock_5_5/'+str(stockno)+'.csv')
#     print(str(stockno)+" complete....")

#     def normalization(df):
#     #{'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}
#       dic = {}
#       for col in df.columns:
#           if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
#           else:
#               mean = statistics.mean(df[str(col)])
#               stdev = statistics.stdev(df[str(col)])
#               if stdev == 0: stdev = 1
#           dic[str(col)+'_mean'] = mean
#           dic[str(col)+'_stdev'] = stdev
#       return dic
#     record = pd.DataFrame()
#     days_before = 30
#     for i in range(days_before, len(new_df)):
#       record = record.append(normalization(new_df[i-days_before:i]), ignore_index=True )
#     record.to_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5(for_price)/'+str(stockno)+'mean&stdev.csv')
#     print(str(stockno)+" mean & stdev complete....")

Load in training / validation / testing data

In [ ]:



pd.options.mode.chained_assignment = None 

train_x = []
test_x = []
val_x = []
val_y = []
train_y = []
test_y = []

for stockno in stockNum:
    x = []
    if int(stockno) == 2227: continue
    df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5(for_price)/'+str(stockno)+'.csv')
    record = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5(for_price)/'+str(stockno)+'mean&stdev.csv')
    df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
    df = df.drop(['Date'], axis=1)
    df = df.drop(['MarginPurchaseLimit'], axis=1)
    df = df.drop(['ShortSaleLimit'], axis=1)
    df = df.dropna(axis=0, how='any')
    df.reset_index(drop=True, inplace=True)
    days_before = 30
    for i in range(days_before, len(df)):
        tmp = df[i-days_before:i]
        for col in df.columns:
          if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
          mean = record[str(col)+'_mean'][i-30]
          stdev = record[str(col)+'_stdev'][i-30]
          def test(s): 
            return (float(s)-mean)/stdev
          tmp[str(col)].map(test)
        x.append( np.array(np.array(tmp).tolist()[0: days_before]).reshape(days_before,len(tmp.columns)))
    y = list(df['y_label'][days_before:]) 
    #train:val:test = 8:1:1
    tr_va = int(len(x)*8/10)
    va_te = int(len(x)*9/10)
    # print("no: ", tr_va, "no: ", va_te, "leng: ", len(y))
    train_x = train_x + x[:tr_va]
    train_y = train_y + y[:tr_va]
    val_x = val_x + x[tr_va: va_te]
    val_y = val_y + y[tr_va: va_te]
    test_x = test_x + x[va_te:]
    test_y = test_y +y[va_te:]
    print(str(stockno)+" loaded....")
    # print(len(train_x), len(val_x), len(test_x))

1101 loaded....
1102 loaded....
1216 loaded....
1301 loaded....
1326 loaded....
1402 loaded....
2002 loaded....
2105 loaded....
2207 loaded....
2301 loaded....
2303 loaded....
2308 loaded....
2317 loaded....
2327 loaded....
2330 loaded....
2352 loaded....
2357 loaded....
2382 loaded....
2395 loaded....
2408 loaded....
2412 loaded....
2454 loaded....
2474 loaded....
2609 loaded....
2610 loaded....
2633 loaded....
2801 loaded....
2823 loaded....
2880 loaded....
2881 loaded....
2882 loaded....
2883 loaded....
2884 loaded....
2885 loaded....
2886 loaded....
2887 loaded....
2888 loaded....
2890 loaded....
2891 loaded....
2892 loaded....
2912 loaded....
3008 loaded....
3045 loaded....
3711 loaded....
4904 loaded....
4938 loaded....
5871 loaded....
5880 loaded....
6505 loaded....
9904 loaded....
9910 loaded....


In [ ]:
start = '2012/01/01'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))
# Price_df = pdr.DataReader(str(2330)+'.TW', 'yahoo', start=start, end=end)
# Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
# Price_df = Price_df.reset_index()
# Price_df = Price_df[-2016:]
# print(Price_df.columns)
# print(Price_df['index'][1612])
# print(Price_df['index'][1814])

In [ ]:
y = [] 
for i in range(len(train_y)):
    y.append(to_onelist(str(train_y[i])))
train_y = y

y = [] 
for i in range(len(test_y)):
    y.append(to_onelist(str(test_y[i])))
test_y = y

y = [] 
for i in range(len(val_y)):
    y.append(to_onelist(str(val_y[i])))
val_y = y

In [ ]:
def count_label_dist(arr):
    count_1 = 0 
    count_0 = 0 
    for i in range(len(arr)):
        if list(arr[i])== list([0,1]): count_1 += 1
        elif list(arr[i])== list([1,0]): count_0 += 1
    print(count_0, count_1)
    return count_0, count_1

c_0, c_1 = count_label_dist(train_y)
c_0, c_1 = count_label_dist(val_y)
c_0, c_1 = count_label_dist(test_y)

37674 42010
4088 5882
5080 4905


In [ ]:
train_x = np.array(train_x).reshape(len(train_x), 30, 23)
test_x  = np.array(test_x).reshape(len(test_x), 30, 23)
val_x   = np.array(val_x).reshape(len(val_x), 30, 23)
train_y = np.array(train_y).reshape(len(train_y),2)
test_y  = np.array(test_y).reshape(len(test_y),2)
val_y   = np.array(val_y).reshape(len(val_y),2)

In [ ]:
# train_x[1]

In [ ]:
def new_multi1_model(input_shape, n_classes):
    model = Sequential()

    model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
    model.add(Conv1D(filters=80,kernel_size=3,padding='valid',activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    #model.add(Dropout(0.2))
    
    model.add(BatchNormalization())
    model.add(Conv1D(filters=80,kernel_size=5,padding='valid',activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    #model.add(Dropout(0.2))
    
    model.add(BatchNormalization())
    model.add(Conv1D(filters=80,kernel_size=7,padding='valid',activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    #model.add(Dropout(0.2))
    
    model.add(BatchNormalization())
    model.add(Conv1D(filters=80,kernel_size=9,padding='valid',activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    #model.add(Dropout(0.2))

    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(500,activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))

    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(40,activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))

    model.add(BatchNormalization())
    model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=10000, decay_rate=0.9)
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
    #model.compile(loss=mycrossentropy, optimizer=Adamax(learning_rate=0.0001), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
def lstm_model(input_shape , n_classes):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
    model.add(LSTM(100,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(50,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(25,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(12,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(6,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(3,return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(90, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(10, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=10000, decay_rate=0.9)
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
def lstm2_model(input_shape , n_classes):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
    model.add(LSTM(100,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(100,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(50,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(50,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(25,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(12,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(6,return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(90, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=10000, decay_rate=0.9)
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
def lstm3_model(input_shape , n_classes):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
    model.add(LSTM(50,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(25,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(12,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(6,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(3,return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(90, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(10, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=10000, decay_rate=0.9)
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
def lstm4_model(input_shape , n_classes):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
    model.add(LSTM(25,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(12,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(6,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(3,return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(90, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(25, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=100000, decay_rate=0.9)
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
def lstm5_model(input_shape , n_classes):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
    model.add(LayerNormalization())
    model.add(LSTM(23,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LayerNormalization())
    model.add(LSTM(2,return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(LayerNormalization())
    model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=50, decay_rate=0.9)
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
def lstm6_model(input_shape , n_classes):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
    model.add(LSTM(50,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(25,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(12,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(6,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(3,return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(90, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(25, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=100000, decay_rate=0.9)
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
# tf.keras.backend.clear_session()
# model = new_multi1_model((30, 23), 2)
model = lstm4_model((30, 23), 2)
callback = EarlyStopping(monitor="val_loss", patience=25, verbose=1, mode="auto")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 30, 23)            92        
_________________________________________________________________
lstm (LSTM)                  (None, 30, 25)            4900      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 25)            100       
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 12)            1824      
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 12)            48        
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 6)             456       
_________________________________________________________________
batch_normalization_3 (Batch (None, 30, 6)             2

In [ ]:
history = model.fit(train_x, train_y , epochs=150, batch_size= 256, validation_data=(val_x, val_y), callbacks=[callback, tensorboard_callback] )

In [ ]:
# %tensorboard --logdir logs
# count = 0
# correct_0 = 0
# correct_1 = 0
# pred_0 = 0
# pred_1 = 0
# for i in range(len(test_x)):
#     prediction = model.predict(test_x[i].reshape(1,30,23))
#     ans = classes[np.argmax(prediction)]
#     if int(ans) == 0 : pred_0 +=1
#     else: pred_1 +=1
#     if int(ans) == int(classes[np.argmax(test_y[i])]): 
#       count +=1
#       if int(ans) == 0 : correct_0 += 1
#       else: correct_1 += 1
#     if i is not 0 and i%1000 == 0 : print("Accuracy: ", count/i)
# print("Accuracy: ", count/len(test_x))
# print(correct_0, correct_1)
# print(pred_0, pred_1)
# print(correct_0/pred_0, correct_1/pred_1)

In [ ]:
prediction_y = []
truth_y = []
for i in range(len(test_y)):
  prediction_y.append(int(classes[np.argmax(model.predict(test_x[i].reshape(1,30,23)))]))
  truth_y.append(int(classes[np.argmax(test_y[i])]))
print("Accuracy: ")
print( 1 - (np.sum(np.logical_xor(np.array(prediction_y), np.array(truth_y))))/len(test_y) )

Accuracy: 
0.8800200300450676


In [ ]:
model.save('/content/drive/My Drive/Program_Trading_Code/lstm4_only_price.h5')

In [ ]:
#選定欲回測的model
from keras.models import load_model
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
model = load_model('/content/drive/My Drive/Program_Trading_Code/lstm4_5_5.h5')

stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002",
            "2105", "2207", #"2227", 
            "2301", "2303", "2308", "2317", "2327",
            "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610",
            "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
            "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711",
            "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
# stockNum = ["2330"]

#設定 股票資料
start = '2018/08/01'
end = '2019/06/3'
# start = '2019/06/4'
# end = '2020/11/11'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [ ]:
wala_wala = 0
wala_wala_our = 0
wala_wala_origin = 0

Dict_vic = {}
Dict_return={}
for stock_num in stockNum:
  #先算出之後回測用的價位
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  #融資表格
  stock_time = str(Price_df.index[0]).split(" ")[0]
  # print(stock_time)
  #load in 融資融券
  url = "https://api.finmindtrade.com/api/v3/data"
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  # 處理三大法人投資情況 視情況加入
  url = "https://api.finmindtrade.com/api/v3/data"
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  # gpy = data.groupby('date').sum()
  # gpy.rename(columns={'buy':'buy_volume', 'sell': 'sell_volume'}, inplace=True)
  # gpy['bs_ratio'] = gpy['buy_volume'] / (gpy['sell_volume']+1)
  # df = gpy
  # new_df = new_df.join(df)
  # new_df = new_df.dropna(axis=0, how='any')
  # new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    # print(columnName)
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 10
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  #更改index 將日期更改為
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  # new_df = new_df.drop(['level_0'], axis=1)
  Price_df = Price_df.reset_index()

  # start evaluaion 
  ###############一般回測################################
  left_money = 10**6
  own_asset = 10**6
  time_slide = 30 
  own_stock = 0 
  money_record = [left_money]*time_slide
  # flag = 0 
  origin_money = 10**6 #紀錄開場進最後出
  origin_hold_stock = 0 

  base_victory = 0 
  count_victory = 0 
  temporary = 0

  for i in range(time_slide, len(new_df)): 
    tmp = new_df[i-time_slide:i]
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model.predict(x.reshape(1,30,23)))])
    price = int(Price_df['Close'][i]*100)/100
    if i == time_slide: #起始買
      origin_hold_stock += int( left_money / ( price * (1 + 0.001425)) ) # 包含手續費可以購買的股數
      origin_money = origin_money - origin_hold_stock*price*(1 + 0.001425)
    elif i == len(new_df)-1: #結束賣
      origin_money = origin_money + origin_hold_stock*price*(1 - 0.001425 - 0.003)
    if predict == 1:
      if own_stock ==0:
        # print("買入")
        own_stock += int( left_money / ( price * (1 + 0.001425)) )
        left_money = left_money - own_stock*price*(1 + 0.001425)
        # print("目前價格： " , price, "日期: ",time_list[i])
        # print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
        temporary =  price

    elif predict == 0 :
      if own_stock > 0:
        # print("賣出")
        left_money += own_stock*price*(1 - 0.001425 - 0.003)
        own_stock = 0
        # print("目前價格： " , price, "日期: ", time_list[i])
        # print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
        if price > temporary:
          count_victory +=1
        base_victory +=1
    money_record.append(left_money+own_stock*price)
  print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(10**6))*100/(10**6),"%")
  print(str(stock_num)+"最後資產 ：", int(origin_money),"報酬率：",(int(origin_money)-(10**6))*100/(10**6),"%（開場進 最後出）")
  print(str(stock_num)+"validation 勝率：", count_victory/base_victory,'(',count_victory,'/',base_victory ,')')
  
  Dict_return[str(stock_num)] = (int(money_record[-1])-(10**6))*100/(10**6)
  Dict_vic[str(stock_num)] = count_victory/base_victory

  wala_wala += 10**6
  wala_wala_our += int(money_record[-1])
  wala_wala_origin += int(origin_money)
print("測試結束")
print("our總投資報酬率：", (wala_wala_our - wala_wala)*100 / wala_wala, "%")
print("origin總投資報酬率：", (wala_wala_origin - wala_wala)*100 / wala_wala, "%")

1101最後資產 ： 913904 報酬率： -8.6096 %
1101最後資產 ： 1111544 報酬率： 11.1544 %（開場進 最後出）
1101validation 勝率： 0.42857142857142855 ( 3 / 7 )
1102最後資產 ： 1101097 報酬率： 10.1097 %
1102最後資產 ： 1247586 報酬率： 24.7586 %（開場進 最後出）
1102validation 勝率： 0.7142857142857143 ( 5 / 7 )
1216最後資產 ： 992065 報酬率： -0.7935 %
1216最後資產 ： 1052713 報酬率： 5.2713 %（開場進 最後出）
1216validation 勝率： 0.3333333333333333 ( 2 / 6 )
1301最後資產 ： 905646 報酬率： -9.4354 %
1301最後資產 ： 976246 報酬率： -2.3754 %（開場進 最後出）
1301validation 勝率： 0.3333333333333333 ( 2 / 6 )
1326最後資產 ： 820529 報酬率： -17.9471 %
1326最後資產 ： 897276 報酬率： -10.2724 %（開場進 最後出）
1326validation 勝率： 0.2 ( 1 / 5 )
1402最後資產 ： 887077 報酬率： -11.2923 %
1402最後資產 ： 919563 報酬率： -8.0437 %（開場進 最後出）
1402validation 勝率： 0.6 ( 3 / 5 )
2002最後資產 ： 904523 報酬率： -9.5477 %
2002最後資產 ： 990474 報酬率： -0.9526 %（開場進 最後出）
2002validation 勝率： 0.5714285714285714 ( 4 / 7 )
2105最後資產 ： 779513 報酬率： -22.0487 %
2105最後資產 ： 849971 報酬率： -15.0029 %（開場進 最後出）
2105validation 勝率： 0.16666666666666666 ( 1 / 6 )
2207最後資產 ： 1756757 報酬率： 75.6757 %
22

In [ ]:
# plt.plot(time_list, money_record)


In [ ]:
pd.options.mode.chained_assignment = None 
dic = {}
from keras.models import load_model
from sklearn.metrics import accuracy_score
model = load_model('/content/drive/My Drive/Program_Trading_Code/lstm4_5_5(1025-1000).h5')
#model = load_model('/content/drive/My Drive/Program_Trading_Code/lstm4_only_price.h5')
TEST_Y = []
TRUTH_Y = []
for stockno in stockNum:
  train_x = []
  test_x = []
  val_x = []
  train_y = []
  test_y = []
  val_y = []
  x = []
  if int(stockno) == 2227: continue
  df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5/'+str(stockno)+'.csv')
  record = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5/'+str(stockno)+'mean&stdev.csv')
  df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
  df = df.drop(['Date'], axis=1)
  df = df.drop(['MarginPurchaseLimit'], axis=1)
  df = df.drop(['ShortSaleLimit'], axis=1)
  df = df.dropna(axis=0, how='any')
  df.reset_index(drop=True, inplace=True)
  days_before = 30
  for i in range(days_before, len(df)):
      tmp = df[i-days_before:i]
      for col in df.columns:
        if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
        mean = record[str(col)+'_mean'][i-days_before]
        stdev = record[str(col)+'_stdev'][i-days_before]
        def test(s): 
          return (float(s)-mean)/stdev
        tmp[str(col)].map(test)
      x.append( np.array(np.array(tmp).tolist()[0: days_before]).reshape(days_before,len(tmp.columns)))
  y = list(df['y_label'][days_before:]) 
  #train:val:test = 8:1:1
  tr_va = int(len(x)*8/10)
  va_te = int(len(x)*9/10)
  # print("no: ", tr_va, "no: ", va_te, "leng: ", len(y))
  train_x = train_x + x[:tr_va]
  train_y = train_y + y[:tr_va]
  val_x = val_x + x[tr_va: va_te]
  val_y = val_y + y[tr_va: va_te]
  test_x = test_x + x[va_te:]
  test_y = test_y + y[va_te:]
  print(str(stockno)+" loaded....")
  # print(len(train_x), len(val_x), len(test_x))
  prediction_y = []
  truth_y = []
  for i in range(len(val_x)):
    prediction_y.append(int(classes[np.argmax(model.predict(val_x[i].reshape(1,30,23)))]))
    #此處無需做argmax 因為load進時沒有做 to_onelist
    truth_y.append(val_y[i])
  TEST_Y += prediction_y
  TRUTH_Y += truth_y
  accuracy = accuracy_score(np.array(prediction_y), np.array(truth_y))
  print("Validation Accuracy: ", accuracy)
  dic[str(stockno)] = accuracy
print("================================total accuracy: ")
print(accuracy_score(np.array(TEST_Y), np.array(TRUTH_Y)))

In [ ]:
# accuracy = 1 - (np.sum(np.logical_xor(np.array(TEST_Y), np.array(TRUTH_Y))))/len(TEST_Y) 
# print(accuracy)

Validation Find 投資報酬率 & 計算權重

In [ ]:
#選定欲回測的model
from keras.models import load_model
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
model = load_model('/content/drive/My Drive/Program_Trading_Code/lstm4_5_5.h5')
#model = load_model('/content/drive/My Drive/Program_Trading_Code/lstm4_only_price.h5')

stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002",
            "2105", "2207", #"2227", 
            "2301", "2303", "2308", "2317", "2327",
            "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610",
            "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
            "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711",
            "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

#設定 股票資料
#start = '2019/05/13'
start = '2018/08/01'
end = '2019/06/03'
#end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [ ]:
wala_wala = 0
wala_wala_our = 0
wala_wala_origin = 0
#asset_at_the_start = 5.1 * 10**7
DIC_ROR = {}
for stock_num in stockNum:
  #先算出之後回測用的價位
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  #融資表格
  stock_time = str(Price_df.index[0]).split(" ")[0]
  # print(stock_time)
  #load in 融資融券
  url = "https://api.finmindtrade.com/api/v3/data"
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  # 處理三大法人投資情況 視情況加入
  url = "https://api.finmindtrade.com/api/v3/data"
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  # gpy = data.groupby('date').sum()
  # gpy.rename(columns={'buy':'buy_volume', 'sell': 'sell_volume'}, inplace=True)
  # gpy['bs_ratio'] = gpy['buy_volume'] / (gpy['sell_volume']+1)
  # df = gpy
  # new_df = new_df.join(df)
  # new_df = new_df.dropna(axis=0, how='any')
  # new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    # print(columnName)
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 10
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  #更改index 將日期更改為
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  # new_df = new_df.drop(['level_0'], axis=1)
  Price_df = Price_df.reset_index()

  # start evaluaion 
  ###############一般回測################################
  left_money = 10**6
  own_asset = 10**6
  time_slide = 30 
  own_stock = 0 
  money_record = [left_money]*time_slide
  origin_money = 10**6 #紀錄開場進最後出
  origin_hold_stock = 0 
  for i in range(time_slide, len(new_df)): 
    tmp = new_df[i-time_slide:i]
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model.predict(x.reshape(1,30,23)))])
    price = int(Price_df['Close'][i]*100)/100
    if i == time_slide: #起始買
      origin_hold_stock += int( left_money / ( price * (1 + 0.001425)) ) # 包含手續費可以購買的股數
      origin_money = origin_money - origin_hold_stock*price*(1 + 0.001425)
    elif i == len(new_df)-1: #結束賣
      origin_money = origin_money + origin_hold_stock*price*(1 - 0.001425 - 0.003)
    if predict == 1:
      if own_stock ==0:
        # print("買入")
        own_stock += int( left_money / ( price * (1 + 0.001425)) )
        left_money = left_money - own_stock*price*(1 + 0.001425)
        # print("目前價格： " , price, "日期: ",time_list[i])
        # print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
    elif predict == 0 :
      if own_stock > 0:
        # print("賣出")
        left_money += own_stock*price*(1 - 0.001425 - 0.003)
        own_stock = 0
        # print("目前價格： " , price, "日期: ", time_list[i])
        # print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
    money_record.append(left_money+own_stock*price)
  print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(10**6))*100/(10**6),"%")
  print(str(stock_num)+"最後資產 ：", int(origin_money),"報酬率：",(int(origin_money)-(10**6))*100/(10**6),"%（開場進 最後出）")
  #wala_wala_weight += asset_at_the_start * DIC[stock_num]
  wala_wala += 10**6
  wala_wala_our += int(money_record[-1])
  wala_wala_origin += int(origin_money)
  DIC_ROR[str(stock_num)] = (int(money_record[-1])-(10**6))*100/(10**6) - (int(origin_money)-(10**6))*100/(10**6)
  print(DIC_ROR[str(stock_num)])
print("測試結束")
print("our總投資報酬率：", (wala_wala_our - wala_wala)*100 / wala_wala, "%")
# print("origin總投資報酬率：", (wala_wala_origin - wala_wala)*100 / wala_wala, "%")

In [ ]:
# Dict_vic 
# Dict_return
# Weights_Dict = {}
#勝率超過五成 且報酬率為正
Weights_Dict = {}
for key in Dict_vic:
  # print(key, '->', DIC_ROR[key])
  if Dict_vic[key] > 0.5 and Dict_return[key] >0:
    print(key, '->', Dict_vic[key],"報酬: ",Dict_return[key] )
    Weights_Dict[key] = Dict_return[key]
WeightSum = sum(Weights_Dict.values())
NewWeightSum = 0
for key in Weights_Dict:
  Weights_Dict[key] /= WeightSum
  print(key, '->', Weights_Dict[key])
  NewWeightSum += Weights_Dict[key]
print(Weights_Dict)
print(NewWeightSum)

1102 -> 0.7142857142857143 報酬:  10.1097
2207 -> 0.75 報酬:  75.6757
2301 -> 0.5714285714285714 報酬:  13.9152
2317 -> 0.6 報酬:  12.161
2610 -> 0.5714285714285714 報酬:  16.8774
2633 -> 1.0 報酬:  44.6408
2880 -> 0.5714285714285714 報酬:  5.6272
2886 -> 0.6666666666666666 報酬:  18.3774
3045 -> 0.5714285714285714 報酬:  2.3242
4904 -> 0.5714285714285714 報酬:  2.2686
5880 -> 0.6 報酬:  1.4831
1102 -> 0.04968880906987752
2207 -> 0.37194332260396756
2301 -> 0.06839270363800704
2317 -> 0.059770874219688075
2610 -> 0.08295180927188255
2633 -> 0.2194079139763384
2880 -> 0.027657484039883953
2886 -> 0.09032425490378224
3045 -> 0.011423358758440838
4904 -> 0.01115008677368509
5880 -> 0.0072893827444469525
{'1102': 0.04968880906987752, '2207': 0.37194332260396756, '2301': 0.06839270363800704, '2317': 0.059770874219688075, '2610': 0.08295180927188255, '2633': 0.2194079139763384, '2880': 0.027657484039883953, '2886': 0.09032425490378224, '3045': 0.011423358758440838, '4904': 0.01115008677368509, '5880': 0.007289382

In [ ]:
# DIC_ROR = {'1101': -8.6096, '1102': 10.1097, '1216': -0.7935, '1301': -9.4354, '1326': -17.9471, '1402': -11.2923, '2002': -9.5477, '2105': -22.0487, '2207': 75.6757, '2301': 13.9152, '2303': -1.3974, '2308': -4.2712, '2317': 12.161, '2327': -8.3967, '2330': -7.3642, '2352': -0.7237, '2357': -24.599, '2382': 13.4822, '2395': 4.3062, '2408': -15.2948, '2412': 0.2369, '2454': -12.0878, '2474': -10.7817, '2609': -5.6983, '2610': 16.8774, '2633': 44.6408, '2801': -4.7183, '2823': -16.7678, '2880': 5.6272, '2881': -15.3308, '2882': -20.0186, '2883': -11.0375, '2884': 21.6132, '2885': -4.1252, '2886': 18.3774, '2887': -6.0796, '2888': -17.4904, '2890': 6.5775, '2891': -14.73, '2892': 2.4412, '2912': -16.2974, '3008': 0.411, '3045': 2.3242, '3711': -20.7536, '4904': 2.2686, '4938': -0.4619, '5871': -1.0963, '5880': 1.4831, '6505': -15.977, '9904': 4.9289, '9910': -14.1742}

In [ ]:
Weights_Dict = {}
for key in DIC_ROR:
  # print(key, '->', DIC_ROR[key])
  if DIC_ROR[key] > 0:
    Weights_Dict[key] = DIC_ROR[key]
WeightSum = sum(Weights_Dict.values())
NewWeightSum = 0
for key in Weights_Dict:
  Weights_Dict[key] /= WeightSum
  print(key, '->', Weights_Dict[key])
  NewWeightSum += Weights_Dict[key]
print(Weights_Dict)
print(NewWeightSum)

2303 -> 0.07721988355903647
2317 -> 0.20061911070022445
2327 -> 0.2216628555227093
2330 -> 0.01960974801043502
2352 -> 0.01265359631722586
2474 -> 0.17906111223453766
2609 -> 0.00676396655344991
2610 -> 0.06483766955454759
2823 -> 0.005151512111623681
2883 -> 0.01771007182127627
2886 -> 0.0013949262067141379
2888 -> 0.0374993334237941
3008 -> 0.07314703694336927
4938 -> 0.08266917704105634
{'2303': 0.07721988355903647, '2317': 0.20061911070022445, '2327': 0.2216628555227093, '2330': 0.01960974801043502, '2352': 0.01265359631722586, '2474': 0.17906111223453766, '2609': 0.00676396655344991, '2610': 0.06483766955454759, '2823': 0.005151512111623681, '2883': 0.01771007182127627, '2886': 0.0013949262067141379, '2888': 0.0374993334237941, '3008': 0.07314703694336927, '4938': 0.08266917704105634}
1.0


In [ ]:
# lstm_4_5_5
Weights_Dict = {'1102': 0.0392674671615576, '2207': 0.2939348412591753, '2301': 0.054048553275221456, '2317': 0.0472349988774842, '2382': 0.05236672164016261, '2395': 0.016725873872726126, '2412': 0.0009201522271257303, '2610': 0.06555414604513213, '2633': 0.17339101536797932, '2880': 0.021856819807859477, '2884': 0.08394864548465104, '2886': 0.0713803526330958, '2890': 0.02554791588822073, '2892': 0.009481957015024621, '3008': 0.0015963806051020478, '3045': 0.009027512901163454, '4904': 0.008811554843636268, '5880': 0.005760564660405955, '9904': 0.019144526434276114}
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002",
            "2105", "2207", "2227", 
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
            "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
Return_Dict = {}
for key in stockNum:
  if key not in Weights_Dict.keys(): Return_Dict[key] = 0.0
  else: Return_Dict[key] = Weights_Dict[key]
Return_Weight_df = pd.DataFrame(columns =['Name', 'val'])
for key in Return_Dict:
  Return_Weight_df = Return_Weight_df.append({'Name': key, 'val': Return_Dict[key]}, ignore_index=True)
Return_Weight_df.to_csv('/content/drive/My Drive/Program_Trading_Code/Return_Weight_df.csv')

In [ ]:
# # TESTRORLSTM455 = {"1102": -23.165001270379722,
# # "2207": 96.55600420568851, "2301": -9.967695810563841, "2317": 0.6622009180339203, "2382": 1.4350504721625053, "2395": 8.554158169004804, "2412": 2.8834341338783886, 
# # "2610": -5.454131307828028, "2633": 9.355966715013592, "2880": -0.9108505606228517, "2884": 14.328480194315958, "2886": 6.500115354557333, "2890": -17.592412842025244, 
# # "2892": 6.584148629664455, "3008": -24.332119510820842, "3045": -15.77035459807336, "4904": -16.856778896328365, "5880": 3.7827719442561625, "9904": -29.27592374683791}
# # lstm_4_5_5
# Weights_Dict = {'1102': 0.0392674671615576, '2207': 0.2939348412591753, '2301': 0.054048553275221456, '2317': 0.0472349988774842, '2382': 0.05236672164016261, '2395': 0.016725873872726126, '2412': 0.0009201522271257303, '2610': 0.06555414604513213, '2633': 0.17339101536797932, '2880': 0.021856819807859477, '2884': 0.08394864548465104, '2886': 0.0713803526330958, '2890': 0.02554791588822073, '2892': 0.009481957015024621, '3008': 0.0015963806051020478, '3045': 0.009027512901163454, '4904': 0.008811554843636268, '5880': 0.005760564660405955, '9904': 0.019144526434276114}
# stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002",
#             "2105", "2207", "2227", 
#             "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
#             "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
# Return_Dict = {}
# for key in stockNum:
#   if key not in Weights_Dict.keys(): Return_Dict[key] = 0.0
#   else: Return_Dict[key] = Weights_Dict[key]
# Return_Weight_df = pd.DataFrame(columns =['Name', 'val','Testing_ROR'])
# for key in Return_Dict:
#   Return_Weight_df = Return_Weight_df.append({'Name': key, 'val': Return_Dict[key]}, ignore_index=True)
# Return_Weight_df.to_csv('/content/drive/My Drive/Program_Trading_Code/Return_Weight_df.csv')

In [ ]:
#選定欲回測的model
from keras.models import load_model
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
model = load_model('/content/drive/My Drive/Program_Trading_Code/lstm4_5_5.h5')
#model = load_model('/content/drive/My Drive/Program_Trading_Code/lstm4_only_price.h5')

stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002",
            "2105", "2207", #"2227", 
            "2301", "2303", "2308", "2317", "2327",
            "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610",
            "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
            "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711",
            "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
# stockNum = ["2330"]

#設定 股票資料
# start = '2019/05/13'
# start = '2018/08/01'
start = '2019/06/03'
end = '2020/11/11'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [ ]:
asset_at_the_start = 19 * 10**6
total_asset_present_value = 0
for stock_num in stockNum:
  if stock_num not in Weights_Dict.keys(): continue
  #先算出之後回測用的價位
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Return_Test_Stock_df = Price_df.copy(deep=True)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  stock_time = str(Price_df.index[0]).split(" ")[0]
  url = "https://api.finmindtrade.com/api/v3/data" #load in 融資融券
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  url = "https://api.finmindtrade.com/api/v3/data" #load in三大法人投資情況
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 10
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  Price_df = Price_df.reset_index()
  Return_Test_Stock_df.reset_index()
  Return_Test_Stock_df = Return_Test_Stock_df[30:]
  # start evaluaion 
  ###############一般回測################################
  y_preds_list=[]
  y_truth_list=[]
  if stock_num not in Weights_Dict.keys(): continue
  left_money = asset_at_the_start * Weights_Dict[stock_num] #10**6
  own_asset = left_money #10**6
  time_slide = 30 
  own_stock = 0 
  money_record = [left_money]*time_slide
  origin_hold_stock = 0 
  print(len(new_df))
  for i in range(time_slide, len(new_df)): 
    tmp = new_df[i-time_slide:i]
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model.predict(x.reshape(1,30,23)))])
    y_truth_list.append(new_df['y_label'][i])
    y_preds_list.append(predict)
    price = int(Price_df['Close'][i]*100)/100
    if predict == 1:
      if own_stock == 0:
        own_stock += int( left_money / ( price * (1 + 0.001425)) )
        left_money = left_money - own_stock*price*(1 + 0.001425)
    elif predict == 0 :
      if own_stock > 0:
        left_money += own_stock*price*(1 - 0.001425 - 0.003)
        own_stock = 0
    money_record.append(left_money+own_stock*price)
  print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(own_asset))*100/(own_asset),"%")
  total_asset_present_value += int(money_record[-1])
  # insert y label and y truth
  Return_Test_Stock_df.insert(6, 'y_pred', y_preds_list)
  Return_Test_Stock_df.insert(7, 'y_true', y_truth_list)
  Return_Test_Stock_df.to_csv('/content/drive/My Drive/Program_Trading_Code/Return_Test_Stock_DF/Return_Test_Stock_df'+str(stock_num)+'.csv')
print("測試結束")
print("our總投資報酬率：", (total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start, "%")

355
1102最後資產 ： 757867 報酬率： -19.724908709301086 %
355
2207最後資產 ： 13796928 報酬率： 95.23246181148458 %
355
2301最後資產 ： 1142155 報酬率： -12.105505624103605 %
355
2317最後資產 ： 1044057 報酬率： -8.06497455753726 %
355
2610最後資產 ： 1640768 報酬率： 4.104071127942479 %
355
2633最後資產 ： 4590985 報酬率： 10.12856605516171 %
355
2880最後資產 ： 504589 報酬率： -3.9778319995548026 %
355
2886最後資產 ： 1799472 報酬率： 4.854507499084951 %
355
3045最後資產 ： 203914 報酬率： -6.049385155276981 %
355
4904最後資產 ： 179010 報酬率： -15.502191699494716 %
355
5880最後資產 ： 139674 報酬率： 0.8489115692947312 %
測試結束
our總投資報酬率： 35.786415789473686 %


In [ ]:
DIC_ROR

{'1101': -19.764000000000003,
 '1102': -14.648900000000001,
 '1216': -6.0648,
 '1301': -7.06,
 '1326': -7.6747,
 '1402': -3.2485999999999997,
 '2002': -8.5951,
 '2105': -7.0458,
 '2207': -21.580299999999994,
 '2301': -12.4837,
 '2303': 14.6255,
 '2308': -31.8462,
 '2317': 37.9974,
 '2327': 41.98310000000001,
 '2330': 3.7141,
 '2352': 2.3966,
 '2357': -8.189599999999999,
 '2382': -0.9771000000000001,
 '2395': -17.3779,
 '2408': -20.7723,
 '2412': -1.8900000000000001,
 '2454': -38.7987,
 '2474': 33.9143,
 '2609': 1.2811000000000003,
 '2610': 12.2803,
 '2633': -28.4765,
 '2801': -14.8565,
 '2823': 0.9757000000000033,
 '2880': -10.6971,
 '2881': -3.1448,
 '2882': -1.6160999999999994,
 '2883': 3.3543000000000003,
 '2884': -0.8599999999999994,
 '2885': -20.2122,
 '2886': 0.26420000000000243,
 '2887': -5.2017,
 '2888': 7.102399999999999,
 '2890': -4.5341,
 '2891': -12.2232,
 '2892': -7.7087,
 '2912': -2.0581999999999994,
 '3008': 13.854099999999999,
 '3045': -5.300000000000001,
 '3711': -3.84